In [1]:
import urllib
import pandas as pd
import numpy as np
import datetime
import string
import pymysql as mdb

In [2]:
maketables=False

In [3]:
topicsfile="/Users/abramvandergeest/Dropbox/insight_work/topic_list.txt"
f=open(topicsfile)
topics=f.readlines()
f.close()

In [4]:
afile="/Users/abramvandergeest/mysql_insightwiki_auth.txt"
a=open(afile)
passwd=a.readline().rstrip()
a.close()
host='localhost'; user='abram.ghost';db='wikidata'
con = mdb.connect(host, user, passwd, db)

####DONE - Also mysql PASSWORD HAS BEEN CHANGED

#I should also create the  database if it doesn't exist

In [5]:
if maketables==True:
    #Makes topics table
    with con:
        cur = con.cursor()
        cur.execute("DROP TABLE IF EXISTS `topics`;")
        str="CREATE TABLE `topics` (`Id` INT PRIMARY KEY AUTO_INCREMENT, `topic_label` VARCHAR(64), `topic_string` VARCHAR(64));"
        cur.execute(str)
    
    #Populate topics table
    with con:
        for topic in topics:
            label="".join(topic.lower().split())
            string=topic.rstrip()
            sql="INSERT INTO `topics` (`topic_label`,`topic_string`) VALUES (%s,%s);"
            cur.execute(sql,(label,string))
        
    #Makes page_views table
    with con:
        cur = con.cursor()
        cur.execute("DROP TABLE IF EXISTS `page_views`;")
        str="CREATE TABLE `page_views` (`Id` INT PRIMARY KEY AUTO_INCREMENT, `topic_id` INT, `count` INT, `date` VARCHAR(64));"
        cur.execute(str)    

In [6]:
def get_unzip(d,fo,fo2):
    wikiurl="http://dumps.wikimedia.org/other/pagecounts-raw/%4d/%4d-%02d/"%(d.year,d.year,d.month)
    gzfile="pagecounts-%4d%02d%02d-%02d0000.gz"%(d.year,d.month,d.day,d.hour)
    print gzfile
    fn=wikiurl+gzfile
    urllib.urlretrieve(fn,fo)
    !gunzip -c $fo>$fo2
    return

def csv_to_reduceddf(fo2):
    allwiki=pd.read_csv(fo2,delim_whitespace=True)
    allwiki.columns=['lang','article','views','size']
    enwiki=allwiki[allwiki['lang']=='en']
    enwiki=enwiki[enwiki['size']>=50000]
    return(enwiki)

In [7]:
#creating a dataframe with all english pages of a large size
print "starting"
year=2015
month=3
day=1
hour=8
fo="/Users/abramvandergeest/wikidata/temp.gz"
fo2="/Users/abramvandergeest/wikidata/temp.txt"
d=datetime.datetime(year,month,day,hour)
#dend=datetime.datetime(2015,11,11,17)
dend=datetime.datetime.now()
print "end date is :",dend
dt=datetime.timedelta(days=+1)

starting
end date is : 2015-09-16 09:28:25.325000


In [8]:
print "entering loop"

while d<datetime.datetime(dend.year,dend.month,dend.day,dend.hour):
    try:
        get_unzip(d,fo,fo2)  #gets the gz file at wikipedia and unzips it
        print "it is unzipped"
        
        enwiki=csv_to_reduceddf(fo2) #takes the unzipped file and pairs it down and puts it into a df
        print "english entries separated"
        
        date='%4d%02d%02d%02d'%(d.year,d.month,d.day,d.hour)
        dic={}
        views=[]
        with con:
            cur = con.cursor()
            sql="SELECT `Id`,`topic_label`,`topic_string` FROM `topics`;"
            cur.execute(sql)
            for row in cur:
                tops=row[2].rstrip().split()
                twiki=enwiki[np.logical_and.reduce([enwiki['article'].str.contains(top,case=False) for top in tops ])==True]
                t=row[2].rstrip()
                dic[t]=twiki['views'].sum(axis=0)
                #print row[0],dic[t],date
                views.append([row[0],dic[t],date])
        print "pageviews sorted"
        
        with con:
            cur = con.cursor()
            for view in views:
                sql="INSERT INTO `page_views` (`topic_id`,`count`,`date`) VALUES (%s,%s,%s);"
                cur.execute(sql,view)
    
        print date
        d+=dt
    except Exception, e:    
        print "Hit an exception but we are continuing: %s",e
        d+=dt

entering loop
pagecounts-20150301-080000.gz
it is unzipped
english entries separated
pageviews sorted
2015030108
pagecounts-20150302-080000.gz
it is unzipped
english entries separated
pageviews sorted
2015030208
pagecounts-20150303-080000.gz
it is unzipped
english entries separated
pageviews sorted
2015030308
pagecounts-20150304-080000.gz
it is unzipped
english entries separated
pageviews sorted
2015030408
pagecounts-20150305-080000.gz
it is unzipped
english entries separated
pageviews sorted
2015030508
pagecounts-20150306-080000.gz
it is unzipped
english entries separated
pageviews sorted
2015030608
pagecounts-20150307-080000.gz
it is unzipped
english entries separated
pageviews sorted
2015030708
pagecounts-20150308-080000.gz
it is unzipped
english entries separated
pageviews sorted
2015030808
pagecounts-20150309-080000.gz
it is unzipped
english entries separated
pageviews sorted
2015030908
pagecounts-20150310-080000.gz
it is unzipped
english entries separated
pageviews sorted
2015031